We cannot work with the text data in machine learning so we need to convert them into numerical vectors, This kernel has all techniques for conversion

> Text data needs to be cleaned and encoded to numerical values before giving them to machine learning models, this process of cleaning and encoding is called as **Text Preprocessing**

In this kernel we are going to see some basic text cleaning steps and techniques for encoding text data. We are going ot see about
1. **Understanding the data** - See what's data is all about. what should be considered for cleaning for data (Punctuations , stopwords etc..).
2. **Basic Cleaning** -We will see what parameters need to be considered for cleaning of data (like Punctuations , stopwords etc..)  and its code.
3. **Techniques for Encoding** - All the popular techniques that are used for encoding that I personally came across.
    *           **Bag of Words**
    *           **Binary Bag of Words**
    *           **Bigram, Ngram**
    *           **TF-IDF**( **T**erm  **F**requency - **I**nverse **D**ocument **F**requency)
    *           **Word2Vec**
    *           **Avg-Word2Vec**
    *           **TF-IDF Word2Vec**

Now, it's time to have fun!

 **Importing Libraries**

In [1]:
import warnings
warnings.filterwarnings("ignore")                     #Ignoring unnecessory warnings

import numpy as np                                  #for large and multi-dimensional arrays
import pandas as pd                                 #for data manipulation and analysis
import nltk                                         #Natural language processing tool-kit

from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.stem import PorterStemmer                 # Stemmer

from sklearn.feature_extraction.text import CountVectorizer          #For Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer          #For TF-IDF
from gensim.models import Word2Vec                                   #For Word2Vec


In [2]:
data_path = "../input/Reviews.csv"
data = pd.read_csv(data_path)
data_sel = data.head(10000)                                #Considering only top 10000 rows

In [3]:
# Shape of our data
data_sel.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

1. **Understanding the data**

Looks like our main objective from the dataset is to predict whether a review is **Positive** or **Negative** based on the Text.
 
 if we see the Score column, it has values 1,2,3,4,5 .  Considering 1, 2 as Negative reviews and 4, 5 as Positive reviews.
 For Score = 3 we will consider it as Neutral review and lets delete the rows that are neutral, so that we can predict either Positive or Negative
 
 HelfulnessNumerator says about number of people found that review usefull and HelpfulnessDenominator is about usefull review count + not so usefull count.
 So, from this we can see that HelfulnessNumerator is always less than or equal to HelpfulnesDenominator.

In [4]:
data_score_removed = data_sel[data_sel['Score']!=3]       #Neutral reviews removed

Converting Score values into class label either Positive or Negative.

In [5]:
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

score_upd = data_score_removed['Score']
t = score_upd.map(partition)
data_score_removed['Score']=t

2. **Basic Cleaning**
 
**Deduplication** means removing duplicate rows, It is necessary to remove duplicates in order to get unbaised results. Checking duplicates based on UserId, ProfileName, Time, Text. If all these values are equal then we will remove those records. (No user can type a review on same exact time for different products.)


We have seen that HelpfulnessNumerator should always be less than or equal to HelpfulnessDenominator so checking this condition and removing those records also.


In [6]:
final_data = data_score_removed.drop_duplicates(subset={"UserId","ProfileName","Time","Text"})

In [7]:
final = final_data[final_data['HelpfulnessNumerator'] <= final_data['HelpfulnessDenominator']]


In [8]:
final_X = final['Text']
final_y = final['Score']

Converting all words to lowercase and removing punctuations and html tags if any

**Stemming**- Converting the words into their base word or stem word ( Ex - tastefully, tasty,  these words are converted to stem word called 'tasti'). This reduces the vector dimension because we dont consider all similar words  

**Stopwords** - Stopwords are the unnecessary words that even if they are removed the sentiment of the sentence dosent change.

Ex -    This pasta is so tasty ==> pasta tasty    ( This , is, so are stopwords so they are removed)

To see all the stopwords see the below code cell.

In [9]:
stop = set(stopwords.words('english')) 
print(stop)

{'myself', 'couldn', 'i', 'while', "you'd", 'are', "hadn't", "haven't", 'have', 'was', 'been', 'yours', 'mightn', 'how', "that'll", "hasn't", "isn't", 'him', 'be', 'all', 'these', 'any', "aren't", 'had', 'didn', 'during', 'ours', "you've", "it's", 'is', 'against', 'just', "should've", 'needn', 'don', 'than', 'hadn', 'by', 'this', 'ourselves', 'my', 'll', 'isn', 'its', 'at', 'themselves', 'that', 'here', 'until', 'off', 'further', 'she', 'each', "shouldn't", "needn't", 'few', 'having', 's', "wouldn't", 'only', "mightn't", 'shouldn', 've', 'when', "you'll", 'should', 'so', 'did', 'why', 'then', "wasn't", 'your', 'on', 'again', 'were', 'it', "shan't", 'm', 'very', 'y', 'doing', 'o', "won't", 'or', 'hasn', 'but', 'will', 'down', 'in', 'aren', 'herself', 'haven', 'not', 'because', "couldn't", 'those', 'the', 'own', 'we', 'for', 'can', 'about', 'does', 'you', 'am', 'them', 'there', 'up', 't', 'whom', 'out', 'between', 'other', 'itself', 'of', 'an', 'a', 'below', 'into', 'what', 'where', 'won

In [10]:
import re
temp =[]
snow = nltk.stem.SnowballStemmer('english')
for sentence in final_X:
    sentence = sentence.lower()                 # Converting to lowercase
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, ' ', sentence)        #Removing HTML tags
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)        #Removing Punctuations
    
    words = [snow.stem(word) for word in sentence.split() if word not in stopwords.words('english')]   # Stemming and removing stopwords
    temp.append(words)
    
final_X = temp    

In [11]:
print(final_X[1])

['product', 'arriv', 'label', 'jumbo', 'salt', 'peanut', 'peanut', 'actual', 'small', 'size', 'unsalt', 'sure', 'error', 'vendor', 'intend', 'repres', 'product', 'jumbo']


In [12]:
sent = []
for row in final_X:
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)

final_X = sent
print(final_X[1])


 product arriv label jumbo salt peanut peanut actual small size unsalt sure error vendor intend repres product jumbo


3. **Techniques for Encoding**

      **BAG OF WORDS**
      
      In BoW we construct a dictionary that contains set of all unique words from our text review dataset.The frequency of the word is counted here. if there are **d** unique words in our dictionary then for every sentence or review the vector will be of length **d** and count of word from review is stored at its particular location in vector. The vector will be highly sparse in such case.
      
      Ex. pasta is tasty and pasta is good
      
     **[0]....[1]............[1]...........[2]..........[2]............[1]..........**             <== Its vector representation ( remaining all dots will be represented as zeroes)
     
     **[a]..[and].....[good].......[is].......[pasta]....[tasty].......**            <==This is dictionary
      .
      
    Using scikit-learn's CountVectorizer we can get the BoW and check out all the parameters it consists of, one of them is max_features =5000 it tells about to consider only top 5000 most frequently repeated words to place in a dictionary. so our dictionary length or vector length will be only 5000
    


   **BINARY BAG OF WORDS**
    
   In binary BoW, we dont count the frequency of word, we just place **1** if the word appears in the review or else **0**. In CountVectorizer there is a parameter **binary = true** this makes our BoW to binary BoW.
   
  

In [13]:
count_vect = CountVectorizer(max_features=5000)
bow_data = count_vect.fit_transform(final_X)
print(bow_data[1])


  (0, 3641)	1
  (0, 2326)	1
  (0, 4734)	1
  (0, 1539)	1
  (0, 4314)	1
  (0, 4676)	1
  (0, 3980)	1
  (0, 4013)	1
  (0, 162)	1
  (0, 3219)	2
  (0, 3770)	1
  (0, 2420)	2
  (0, 2493)	1
  (0, 332)	1
  (0, 3432)	2


 **Drawbacks of BoW/ Binary BoW**
 
 Our main objective in doing these text to vector encodings is that similar meaning text vectors should be close to each other, but in some cases this may not possible for Bow
 
For example, if we consider two reviews **This pasta is very tasty** and **This pasta is not tasty** after stopwords removal both sentences will be converted to **pasta tasty** so both giving exact same meaning.

The main problem is here we are not considering the front and back words related to every word, here comes Bigram and Ngram techniques.

**BI-GRAM BOW**

Considering pair of words for creating dictionary is Bi-Gram , Tri-Gram means three consecutive words so as NGram.

CountVectorizer has a parameter **ngram_range** if assigned to (1,2) it considers Bi-Gram BoW

But this massively increases our dictionary size 

In [14]:
final_B_X = final_X

In [15]:
count_vect = CountVectorizer(ngram_range=(1,2))
Bigram_data = count_vect.fit_transform(final_B_X)
print(Bigram_data[1])

  (0, 143171)	1
  (0, 151696)	1
  (0, 95087)	1
  (0, 196648)	1
  (0, 60866)	1
  (0, 177168)	1
  (0, 193567)	1
  (0, 164722)	1
  (0, 165627)	1
  (0, 4021)	1
  (0, 133855)	1
  (0, 133898)	1
  (0, 155987)	1
  (0, 97865)	1
  (0, 100490)	1
  (0, 11861)	1
  (0, 142800)	1
  (0, 151689)	1
  (0, 95076)	1
  (0, 196632)	1
  (0, 60852)	1
  (0, 177092)	1
  (0, 193558)	1
  (0, 164485)	1
  (0, 165423)	1
  (0, 3831)	1
  (0, 133854)	2
  (0, 155850)	1
  (0, 97859)	2
  (0, 100430)	1
  (0, 11784)	1
  (0, 142748)	2


**TF-IDF**

**Term Frequency -  Inverse Document Frequency** it makes sure that less importance is given to most frequent words and also considers less frequent words.

**Term Frequency** is number of times a **particular word(W)** occurs in a review divided by totall number of words **(Wr)** in review. The term frequency value ranges from 0 to 1.

**Inverse Document Frequency** is calculated as **log(Total Number of Docs(N) / Number of Docs which contains particular word(n))**. Here Docs referred as Reviews.


**TF-IDF** is **TF * IDF** that is **(W/Wr)*LOG(N/n)**


 Using scikit-learn's tfidfVectorizer we can get the TF-IDF.

So even here we get a TF-IDF value for every word and in some cases it may consider different meaning reviews as similar after stopwords removal. so to over come we can use BI-Gram or NGram.

In [16]:
final_tf = final_X
tf_idf = TfidfVectorizer(max_features=5000)
tf_data = tf_idf.fit_transform(final_tf)
print(tf_data[1])

  (0, 3432)	0.1822092004981035
  (0, 332)	0.1574317775964303
  (0, 2493)	0.18769649750089953
  (0, 2420)	0.5671119742041831
  (0, 3770)	0.1536626385509959
  (0, 3219)	0.3726548417697838
  (0, 162)	0.14731616688674187
  (0, 4013)	0.14731616688674187
  (0, 3980)	0.14758995053747803
  (0, 4676)	0.2703170210936338
  (0, 4314)	0.14376924933112933
  (0, 1539)	0.2676489579732629
  (0, 4734)	0.22110622670603633
  (0, 2326)	0.25860104128863787
  (0, 3641)	0.27633136515735446


so to actually overcome the problem of semantical reviews having close distance we have Word2Vec

**Word2Vec**


Word2Vec actually takes the semantic meaning of the words and their relationships between other words. it learns all the internal relationships between the words.It represents the word in dense vector form.

Using **Gensim's** library we have Word2Vec which takes parameters like **min_count = 5** considers only if word repeats more than 5 times in entire data.
**size = 50** gives a vector length of size 50 and **workers** are cores to run this.


**Average Word2Vec**

Compute the Word2vec of each of the words and add the vectors of each words of the sentence and divide the vector with the number of words of the sentence.Simply Averaging the Word2Vec of all words.

In [17]:
w2v_data = final_X

In [18]:
splitted = []
for row in w2v_data: 
    splitted.append([word for word in row.split()])     #splitting words

In [19]:
train_w2v = Word2Vec(splitted,min_count=5,size=50, workers=4)

In [20]:
avg_data = []
for row in splitted:
    vec = np.zeros(50)
    count = 0
    for word in row:
        try:
            vec += train_w2v[word]
            count += 1
        except:
            pass
    avg_data.append(vec/count)
    

In [21]:
print(avg_data[1])

[ 0.10633459  0.13809241 -0.31903746 -0.13181521 -0.2148949  -0.12497693
 -0.04080586  0.62327078  0.08554989 -0.05664759  0.2866584   0.01426788
  0.54800349  0.3330884  -0.09435926  0.02620567 -0.17580219 -0.13678748
 -0.14346219  0.28110569  0.32539288  0.24864704 -0.55982462  0.4809519
 -0.06811121  0.13659069  0.09517061 -0.05140536  0.30564097  0.36861236
  0.34775986  0.72079292 -0.43444114  0.13538126 -0.03166062 -0.15985959
  0.30940363 -0.04276447 -0.08671832  0.36376539  0.30648156 -0.2535723
 -0.39730029  0.03517909  0.06334804 -0.07221556  0.22585394 -0.44916547
 -0.43936174  0.0361915 ]


**TF-IDF WORD2VEC**

in TF-IDF Word2Vec the Word2Vec value of each word is multiplied by the tfidf value of that word and summed up and then divided by the sum of the tfidf values of the sentence.

Something like  

                        V = ( t(W1)*w2v(W1) + t(W2)*w2v(W2) +.....+t(Wn)*w2v(Wn))/(t(W1)+t(W2)+....+t(Wn))

In [22]:
tf_w_data = final_X
tf_idf = TfidfVectorizer(max_features=5000)
tf_idf_data = tf_idf.fit_transform(tf_w_data)

In [23]:
tf_w_data = []
tf_idf_data = tf_idf_data.toarray()
i = 0
for row in splitted:
    vec = [0 for i in range(50)]
    
    temp_tfidf = []
    for val in tf_idf_data[i]:
        if val != 0:
            temp_tfidf.append(val)
    
    count = 0
    tf_idf_sum = 0
    for word in row:
        try:
            count += 1
            tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
            vec += (temp_tfidf[count-1] * train_w2v[word])
        except:
            pass
    vec = (float)(1/tf_idf_sum) * vec
    tf_w_data.append(vec)
    i = i + 1

print(tf_w_data[1])
    

[ 0.06241824  0.08504758 -0.31619775  0.16886339 -0.28298455 -0.10965029
  0.09520426  0.80833275  0.06970318  0.09230006  0.37296815 -0.05549585
  0.51006592  0.43734032 -0.11657764 -0.13956295 -0.02913555 -0.03426978
 -0.1072823   0.31539926  0.33714126  0.34347097 -0.49499545  0.56279667
 -0.47215601  0.12216961  0.15942837 -0.28091281  0.33675403  0.17388769
  0.46250709  0.92158335 -0.65545409  0.15551585 -0.11994225 -0.18773574
  0.47291627 -0.12163895 -0.04144927  0.21858701  0.3434296  -0.41198155
 -0.50600955  0.15004531 -0.02317128 -0.0503962   0.22449031 -0.46675279
 -0.50055964  0.01182543]


**Conclusion**

Throughout this kernel we have seen different techniques for encoding text data into numerical vectors. But which technique is appropriate for our machine learning model depends on the structure of the data and the objective of our model. 